In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
def query(time_range):
    query = 'binderhub_launch_time_seconds_count{}[{}]'
    query_selectors = "{status='success'}"
    query = query.format(query_selectors, time_range)
    resp = requests.get('https://notebooks.gesis.org/prometheus/api/v1/query', params={'query': query})
    data = resp.json()['data']['result']
    return data

In [3]:
def makedf(data):
    df = {'name': [], 'org': [], 'provider': [], 'builds': [], 'url': []}
    for container in data:
        if 'repo' not in container['metric']:
            continue
        url = container['metric']['repo']
        pref, org, repo = container['metric']['repo'].rsplit('/', 2)
        provider = pref.replace('https://', '')
        counts = [int(ii[1]) for ii in container['values']]
        builds = np.max(counts)
        df['name'].append(repo)
        df['org'].append(org)
        df['provider'].append(provider)
        df['builds'].append(builds)
        df['url'].append(url)
    df = pd.DataFrame(df)
    df = df.drop_duplicates(['name'])
    df = df.groupby(['name', 'org', 'provider','url']).sum().reset_index().sort_values('builds', ascending=False)
    df['log_builds'] = df['builds'].apply(np.log)
    return df

In [4]:
def query2(time_range):
    query = 'sum(max_over_time(binderhub_launch_time_seconds_count{}[{}])) by (repo)'
    query_selectors = "{status='success'}"
    query = query.format(query_selectors, time_range)
    resp = requests.get('https://notebooks.gesis.org/prometheus/api/v1/query', params={'query': query})
    data = resp.json()['data']['result']
    return data

In [5]:
def makedf2(data):
    df = {'name': [], 'org': [], 'provider': [], 'builds': [], 'url': []}
    for container in data:
#         if 'repo' not in container['metric']:
#             continue
        repo_url = container['metric']['repo']
        pref, org, repo = container['metric']['repo'].rsplit('/', 2)
        provider = pref.replace('https://', '')
#         counts = [int(ii[1]) for ii in container['values']]
#         builds = np.max(counts)
        df['name'].append(repo)
        df['org'].append(org)
        df['provider'].append(provider)
        df['builds'].append(int(container['value'][1]))
        df['url'].append(repo_url)
    df = pd.DataFrame(df)
    df = df.drop_duplicates(['name'])
    df = df.groupby(['name', 'org', 'provider','url']).sum().reset_index().sort_values('builds', ascending=False)
    df['log_builds'] = df['builds'].apply(np.log)
    return df

In [6]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)

# The most popular repositories in last hour

In [7]:
data = query('1h')
df = makedf(data)
data2 = query2('1h')
df2 = makedf2(data2)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
16,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,12,2.48491
3,binder-stats,gesiscss,github.com,https://github.com/gesiscss/binder-stats,7,1.94591
6,flow,gesiscss,github.com,https://github.com/gesiscss/flow,6,1.79176
13,ptm,gesiscss,github.com,https://github.com/gesiscss/ptm,6,1.79176
10,jupyter-stacks,binder-examples,github.com,https://github.com/binder-examples/jupyter-stacks,3,1.09861
2,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,3,1.09861
15,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,3,1.09861
12,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,2,0.693147
14,remote_storage,binder-examples,github.com,https://github.com/binder-examples/remote_storage,1,0
11,jupyterlab,binder-examples,github.com,https://github.com/binder-examples/jupyterlab,1,0


In [8]:
df2.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
16,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,12,2.48491
3,binder-stats,gesiscss,github.com,https://github.com/gesiscss/binder-stats,7,1.94591
6,flow,gesiscss,github.com,https://github.com/gesiscss/flow,7,1.94591
13,ptm,gesiscss,github.com,https://github.com/gesiscss/ptm,6,1.79176
10,jupyter-stacks,binder-examples,github.com,https://github.com/binder-examples/jupyter-stacks,3,1.09861
2,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,3,1.09861
15,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,3,1.09861
12,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,2,0.693147
14,remote_storage,binder-examples,github.com,https://github.com/binder-examples/remote_storage,1,0
11,jupyterlab,binder-examples,github.com,https://github.com/binder-examples/jupyterlab,1,0


In [9]:
df == df2

,name,org,provider,url,builds,log_builds
16,True,True,True,True,True,True
3,True,True,True,True,True,True
6,True,True,True,True,False,False
13,True,True,True,True,True,True
10,True,True,True,True,True,True
2,True,True,True,True,True,True
15,True,True,True,True,True,True
12,True,True,True,True,True,True
14,True,True,True,True,True,True
11,True,True,True,True,True,True


# The most popular repositories of the day 

In [10]:
data = query('1d')
df = makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
16,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,12,2.48491
3,binder-stats,gesiscss,github.com,https://github.com/gesiscss/binder-stats,7,1.94591
6,flow,gesiscss,github.com,https://github.com/gesiscss/flow,6,1.79176
13,ptm,gesiscss,github.com,https://github.com/gesiscss/ptm,6,1.79176
10,jupyter-stacks,binder-examples,github.com,https://github.com/binder-examples/jupyter-stacks,3,1.09861
2,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,3,1.09861
15,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,3,1.09861
12,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,2,0.693147
14,remote_storage,binder-examples,github.com,https://github.com/binder-examples/remote_storage,1,0
11,jupyterlab,binder-examples,github.com,https://github.com/binder-examples/jupyterlab,1,0


# The most popular repositories of the last week

In [11]:
data = query('7d')
df = makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
20,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,17,2.83321
3,RStan-Binder,arnim,github.com,https://github.com/arnim/RStan-Binder,10,2.30259
2,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,8,2.07944
7,flow,gesiscss,github.com,https://github.com/gesiscss/flow,7,1.94591
4,binder-stats,gesiscss,github.com,https://github.com/gesiscss/binder-stats,7,1.94591
19,tidyverse,tidyverse,github.com,https://github.com/tidyverse/tidyverse,4,1.38629
13,jupyterlab-demo,jupyterlab,github.com,https://github.com/jupyterlab/jupyterlab-demo,4,1.38629
0,CSSproject,nadjalc,github.com,https://github.com/nadjalc/CSSproject,4,1.38629
11,jupyter-stacks,binder-examples,github.com,https://github.com/binder-examples/jupyter-stacks,3,1.09861
12,jupyterlab,binder-examples,github.com,https://github.com/binder-examples/jupyterlab,3,1.09861


# The most popular repositories of the last 30 days

In [12]:
data = query('30d')
df = makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
38,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,69,4.23411
15,flow,gesiscss,github.com,https://github.com/gesiscss/flow,25,3.21888
4,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,24,3.17805
7,YoutubeScrapingEmojis,JuKo007,github.com,https://github.com/JuKo007/YoutubeScrapingEmojis,18,2.89037
3,GitHub_traffic_crawler,gesiscss,github.com,https://github.com/gesiscss/GitHub_traffic_crawler,12,2.48491
32,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,9,2.19722
33,scientific-python-lectures,jrjohansson,github.com,https://github.com/jrjohansson/scientific-python-lectures,9,2.19722
24,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,7,1.94591
11,conda,binder-examples,github.com,https://github.com/binder-examples/conda,7,1.94591
5,RStan-Binder,arnim,github.com,https://github.com/arnim/RStan-Binder,6,1.79176


# The most popular repositories of last 60 days

In [13]:
data = query('60d')
df = makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
37,gesis-meta-analysis-2018,berndweiss,github.com,https://github.com/berndweiss/gesis-meta-analysis-2018,152,5.02388
35,flow,gesiscss,github.com,https://github.com/gesiscss/flow,25,3.21888
15,YoutubeScrapingEmojis,JuKo007,github.com,https://github.com/JuKo007/YoutubeScrapingEmojis,18,2.89037
6,GitHub_traffic_crawler,gesiscss,github.com,https://github.com/gesiscss/GitHub_traffic_crawler,12,2.48491
71,scientific-python-lectures,jrjohansson,github.com,https://github.com/jrjohansson/scientific-python-lectures,9,2.19722
70,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,9,2.19722
27,conda,binder-examples,github.com,https://github.com/binder-examples/conda,7,1.94591
29,convert2geojson,computational-antiquity,github.com,https://github.com/computational-antiquity/convert2geojson,5,1.60944
49,jupyterlab-demo,jupyterlab,github.com,https://github.com/jupyterlab/jupyterlab-demo,4,1.38629
51,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,4,1.38629
